In [ ]:
double default_dcs_pair_production(
    double Z, double A_, double mass, double K, double q)
{
/*
 * Coefficients for the Gaussian quadrature from:
 * https://pomax.github.io/bezierinfo/legendre-gauss.html.
 */

        

### Define Coefficients for the Gaussian quadrature

In [ ]:
#define N_GQ 8
        const double xGQ[N_GQ] = { 0.01985507, 0.10166676, 0.2372338,
                0.40828268, 0.59171732, 0.7627662, 0.89833324, 0.98014493 };
        const double wGQ[N_GQ] = { 0.05061427, 0.11119052, 0.15685332,
                0.18134189, 0.18134189, 0.15685332, 0.11119052, 0.05061427 };

 

In [ ]:
       /*  Check the bounds of the energy transfer. */
        if (q <= 4. * ELECTRON_MASS) return 0.;
        const double sqrte = 1.6487212707;
        const double Z13 = pow(Z, 1. / 3.);
        if (q >= K + mass * (1. - 0.75 * sqrte * Z13)) return 0.;

 

In [ ]:
       /*  Precompute some constant factors for the integration. */
        const double nu = q / (K + mass);


In [ ]:
        const double r = mass / ELECTRON_MASS;

\begin{equation}
r = \frac{M_\mu}{m_e}
\end{equation}

In [ ]:
        const double beta = 0.5 * nu * nu / (1. - nu);
        

\begin{equation}
\beta = \frac{\nu^{2}}{2(1 - \nu)}
\end{equation}

In [ ]:
        const double xi_factor = 0.5 * r * r * beta;


\begin{equation}
\text{xi_factor} = \frac{M_\mu ^{2} \nu^{2}}{4m_e^{2}(1 - \nu)}
\end{equation}

In [ ]:
        const double cL = 2. * sqrte * ELECTRON_MASS * AZ13;

\begin{equation}
cl = 2\sqrt{e} m_e \frac{A}{Z^{1/3}}
\end{equation}

In [ ]:
        const double cLe = 2.25 * Z13 * Z13 / (r * r);

\begin{equation}
cle = \frac{2.25 Z^{2/3}m_e^{2}}{M_\mu^{2}}
\end{equation}

In [ ]:
       /*  Compute the bound for the integral. */
        const double gamma = 1. + K / mass;
        const double x0 = 4. * ELECTRON_MASS / q;
        const double x1 = 6. / (gamma * (gamma - q / mass));
        const double argmin =
            (x0 + 2. * (1. - x0) * x1) / (1. + (1. - x1) * sqrt(1. - x0));
        if ((argmin >= 1.) || (argmin <= 0.)) return 0.;
        const double tmin = log(argmin);



In [ ]:
        /*  Compute the integral over t = ln(1-rho). */
        double I = 0.;
        int i;
        for (i = 0; i < 8; i++) {
                const double eps = exp(xGQ[i] * tmin);
                const double rho = 1. - eps;
                const double rho2 = rho * rho;
                const double rho21 = eps * (2. - eps);
                const double xi = xi_factor * rho21;
                const double xi_i = 1. / xi;

                /* Compute the e-term. */
                double Be;
                if (xi >= 1E+03)
                        Be =
                            0.5 * xi_i * ((3 - rho2) + 2. * beta * (1. + rho2));
                else
                        Be = ((2. + rho2) * (1. + beta) + xi * (3. + rho2)) *
                                log(1. + xi_i) +
                            (rho21 - beta) / (1. + xi) - 3. - rho2;
                const double Ye = (5. - rho2 + 4. * beta * (1. + rho2)) /
                    (2. * (1. + 3. * beta) * log(3. + xi_i) - rho2 -
                                      2. * beta * (2. - rho2));
 

In [ ]:
                const double xe = (1. + xi) * (1. + Ye);                

\begin{equation}
xe = (1 + \xi)(1 + Y_e)
\end{equation}

In [ ]:
                const double cLi = cL / rho21;

In [ ]:
const double Le = log(AZ13 * sqrt(xe) * q / (q + cLi * xe)) - 0.5 * log(1. + cLe * xe);
                

\begin{equation}
Le = \ln(\frac{}{}) 
\end{equation}

In [ ]:
                double Phi_e = Be * Le;
                if (Phi_e < 0.) Phi_e = 0.;

                /* Compute the mu-term. */
                double Bmu;
                if (xi <= 1E-03)
                        Bmu = 0.5 * xi * (5. - rho2 + beta * (3. + rho2));
                else
                        Bmu = ((1. + rho2) * (1. + 1.5 * beta) -
                                  xi_i * (1. + 2. * beta) * rho21) *
                                log(1. + xi) +
                            xi * (rho21 - beta) / (1. + xi) +
                            (1. + 2. * beta) * rho21;
                const double Ymu = (4. + rho2 + 3. * beta * (1. + rho2)) /
                    ((1. + rho2) * (1.5 + 2. * beta) * log(3. + xi) + 1. -
                                       1.5 * rho2);
                const double xmu = (1. + xi) * (1. + Ymu);
                const double Lmu =
                    log(r * AZ13 * q / (1.5 * Z13 * (q + cLi * xmu)));
                double Phi_mu = Bmu * Lmu;
                if (Phi_mu < 0.) Phi_mu = 0.;

                /* Update the t-integral. */
                I -= (Phi_e + Phi_mu / (r * r)) * (1. - rho) * wGQ[i] * tmin;
        }

### Atomic electrons form factor.
ζ(Z) ≈ 1 takes into account the pair production in collisions with
electrons. 

In [ ]:
        double zeta;
        if (gamma <= 35.)
                zeta = 0.;
        else {
                double gamma1, gamma2;
                if (Z == 1.) {
                        gamma1 = 4.4E-05;
                        gamma2 = 4.8E-05;
                } else {
                        gamma1 = 1.95E-05;
                        gamma2 = 5.30E-05;
                }
                zeta = 0.073 * log(gamma / (1. + gamma1 * gamma * Z13 * Z13)) -
                    0.26;
                if (zeta <= 0.)
                        zeta = 0.;
                else {
                        zeta /=
                            0.058 * log(gamma / (1. + gamma2 * gamma * Z13)) -
                            0.14;
                }
        }


### Gather the results and return the macroscopic DCS.

The full muon's energy


In [ ]:
        const double E = K + mass; 

In [ ]:
        const double dcs = 1.794664E-34 * Z * (Z + zeta) * (E - q) * I /(q * E);

\begin{equation}
\frac{\text{d}\sigma^{p}}{\text{d}\nu}(E, \nu) = \alpha^{2} \frac{4}{3\pi} r_e^{2} Z(Z + \xi(Z))(\frac{1 - \nu}{E\nu}) \int_{p}[\Phi_e + (m_e/M_\mu)^{2}\Phi_\mu]dp
\end{equation}

re = 2.8179403262e-15m - Classical electron radius

𝛼=0.0072973525628(6)−fine structure constant

\begin{equation}
\alpha^{2} \frac{4}{3\pi} r_e^{2} = 1.7946638150444265e-34
\end{equation}

In [11]:
import numpy as np
re =  2.8179403262E-15
a = 0.0072973525628
res = 1.794664E-34
out = (4*a*a*re*re)/(3*np.pi)
print(out)
print(res/out)

1.7946638150444265e-34
1.0000001030586185


\begin{equation}
\nu = \frac{q}{E}
\end{equation}

\begin{equation}
\frac{\text{d}\sigma^{p}}{\text{d}\nu}(E, \nu) = \alpha^{2} \frac{4}{3\pi} r_e^{2} Z(Z + \xi(Z))(\frac{E - q}{Eq}) \int_{p}[\Phi_e + (m_e/M_\mu)^{2}\Phi_\mu]dp
\end{equation}

In [ ]:
        return (dcs < 0.) ? 0. : dcs;